# Datenbanken in Python 

[SQLite-Docu](https://www.sqlite.org/lang.html) <br>
Web-Client: [Sqlite Browser](https://sqliteonline.com/)    <br>
Lokaler-Client: [DB Browser for SQLite](https://sqlitebrowser.org/)

#### Tabellen anlegen und mit Daten füllen

In [2]:
import sqlite3 as sq

In [8]:
# Eine Verbindung zur Datenbank aufbauen. 
# Die Datei wird neu angelegt, falls sie noch nicht existiert
connection = sq.connect("lagerverwaltung.db")       
connection.close()

In [3]:
# Eine Datenbank anlegen, ohne sie abzuspeichern
connection = sq.connect(":memory:")       
connection.close()

Um mit einer verbunden Datenbank zu arbeiten, wird ein Cursor benötigt. Den kann man sich als einen Zeiger auf die aktuelle Bearbeitungsposition vorstellen. Es kann mehrere Cursor geben.

SQL-Anweisungen sind nicht case-sensitiv. Es ist aber üblich, SQL Schlüsselworte mit Großbuchstaben zu schreiben.

Beim Anlegen einer Tabelle muss der Datentyp der festgelegt werden.

Python - Sqlite3
* None - NULL
* int - INTEGER
* float - REAL
* str - TEXT
* bytes - BLOB

In [ ]:
# Tabellen in einer Datenbank anlegen
connection = sq.connect("lagerverwaltung.db")  
cursor = connection.cursor()
cursor.execute("""
CREATE TABLE lager (fachnummer INTEGER, seriennummer INTEGER,
komponente TEXT, lieferant TEXT, reserviert INTEGER)
""")

cursor.execute("""
CREATE TABLE lieferanten (kurzname TEXT, name TEXT, telefonnummer TEXT)
""")

cursor.execute(
"""CREATE TABLE kunden (kundennummer INTEGER, name TEXT, anschrift TEXT)
""")
connection.close()


In [7]:
# Daten in eine Tabelle einfügen
connection = sq.connect("lagerverwaltung.db")  
cursor = connection.cursor()
cursor.execute("""
INSERT INTO lager VALUES (1, 26071987, 'Grafikkarte Typ 1', 'FC', 0)
""")
connection.commit()    # erst durch commit werden die Daten 
connection.close()

**Transaktionen** sind Ketten von Operationen, die vollständig ausgeführt werden müssen, damit
die Konsistenz der Datenbank erhalten bleibt. Um die Datenbank transaktionssicher zu machen, werden kritische Anweisungen
mit einer try-except-Anweisung umgeben. Der **rollback** macht alles bis zum letzten **commit** rückgängig.

In [8]:
# Transaktionssicherheit
connection = sq.connect("lagerverwaltung.db")  
try:
    cursor = connection.cursor()
    cursor.execute("""
    INSERT INTO lager VALUES (1, 26071997, 'Grafikkarte Typ 2', 'FC', 0)
    """)
    # Hier können weitere Datenbankmanipulationen stehen
    connection.commit()
except:
    print("Ein Problem trat auf -> Rollback")
    connection.rollback()
connection.close()

Häufig ist es einfacher, die einzufügenden Daten unabhängig vom SQL-Statement zu generieren.


In [9]:
connection = sq.connect("lagerverwaltung.db")  
cursor = connection.cursor()

daten = ("DR", "Danger Electronics","666") 
sql = "INSERT INTO lieferanten VALUES (?,?,?)"
cursor.execute(sql,daten)

connection.commit()     
connection.close()

Wir füllen die Tabelle lager neu:

In [17]:
daten = \
((1, "2607871987", "Grafikkarte Typ 1", "FC", 0),
(2, "19870109", "Prozessor Typ 13", "LPE", 57),
(10, "06198823", "Netzteil Typ 3", "FC", 0),
(25, "11198703", "LED-Lüfter", "FC", 57),
(26, "19880105", "Festplatte 128 GB", "LPE", 12))

connection = sq.connect("lagerverwaltung.db")  
cursor = connection.cursor()
cursor.execute("DROP TABLE IF EXISTS lager")
cursor.execute("""
CREATE TABLE lager (fachnummer INTEGER, seriennummer INTEGER,
komponente TEXT, lieferant TEXT, reserviert INTEGER)
""")

for d in daten:
    sql = "INSERT INTO lager VALUES (?,?,?,?,?)"
    cursor.execute(sql,d)

connection.commit()
connection.close()

Die Methode **executemany** ist schneller als das INSERT in der Schleife. **Dies ist unsere bevorzugte Art, eine Tabelle mit Daten zu füllen:**

In [21]:
lieferanten = \
(("FC", "FiboComputing Inc.", "011235813"),
("LPE", "LettgenPetersErnesti", "026741337"),
("GC", "Golden Computers", "016180339"))

kunden = \
((12, "Heinz Elhurg","Turnhallenstr. 1, 3763 Sporthausen"),
(57, "Markus Altbert","Kämperweg 24, 2463 Duisschloss"),
(64, "Steve Apple","Podmacstr 2, 7467 Iwarhausen"))

connection = sq.connect("lagerverwaltung.db")  
cursor = connection.cursor()

cursor.execute("DROP TABLE IF EXISTS lieferanten")
cursor.execute("DROP TABLE IF EXISTS kunden")

cursor.execute("""
CREATE TABLE lieferanten (kurzname TEXT, name TEXT, telefonnummer TEXT)
""")

cursor.execute("""
CREATE TABLE kunden (kundennummer INTEGER, name TEXT, anschrift TEXT)
""")

cursor.executemany("INSERT INTO lieferanten VALUES (?,?,?)",lieferanten)
cursor.executemany("INSERT INTO kunden VALUES (?,?,?)",kunden)

connection.commit()
connection.close()

#### Abfragen

Das Resultat von **fetchall** ist eine Liste von Tupel.

In [26]:
connection = sq.connect("lagerverwaltung.db")  
cursor = connection.cursor()

sql = "SELECT fachnummer, komponente FROM lager"

cursor.execute(sql)
data = cursor.fetchall()
connection.close()
print(data)


[(1, 'Grafikkarte Typ 1'), (2, 'Prozessor Typ 13'), (10, 'Netzteil Typ 3'), (25, 'LED-Lüfter'), (26, 'Festplatte 128 GB')]


Mit der **WHERE**-Klausel können wir Bedingungen formulieren.

In [23]:
connection = sq.connect("lagerverwaltung.db")  
cursor = connection.cursor()

sql = "SELECT fachnummer, komponente FROM lager WHERE reserviert = 0"

cursor.execute(sql)
data = cursor.fetchall()
connection.close()
print(data)

[(1, 'Grafikkarte Typ 1'), (10, 'Netzteil Typ 3')]


Komplexe Bedingungen können mit **AND** und **OR** formuliert werden

In [25]:
connection = sq.connect("lagerverwaltung.db")  
cursor = connection.cursor()

sql = "SELECT fachnummer, komponente FROM lager WHERE reserviert!=0 AND lieferant='FC'"

cursor.execute(sql)
data = cursor.fetchall()
connection.close()
print(data)

[(25, 'LED-Lüfter')]


Mit dem * erhält man alle Datenfelder

In [27]:
connection = sq.connect("lagerverwaltung.db")  
cursor = connection.cursor()

sql = "SELECT * FROM kunden"

cursor.execute(sql)
data = cursor.fetchall()
connection.close()
print(data)

[(12, 'Heinz Elhurg', 'Turnhallenstr. 1, 3763 Sporthausen'), (57, 'Markus Altbert', 'Kämperweg 24, 2463 Duisschloss'), (64, 'Steve Apple', 'Podmacstr 2, 7467 Iwarhausen')]


Bei Abfragen über mehrere Tabellen nutzen wir die Punkt-Notation

In [28]:
connection = sq.connect("lagerverwaltung.db")  
cursor = connection.cursor()

sql = """
SELECT lager.fachnummer, lager.komponente, lieferanten.name
FROM lager, lieferanten
WHERE lieferanten.telefonnummer='011235813' AND
lager.lieferant=lieferanten.kurzname 
""" 

cursor.execute(sql)
data = cursor.fetchall()
connection.close()
print(data)

[(1, 'Grafikkarte Typ 1', 'FiboComputing Inc.'), (10, 'Netzteil Typ 3', 'FiboComputing Inc.'), (25, 'LED-Lüfter', 'FiboComputing Inc.')]


Liefert eine Abfrage sehr große Datenmengen, dann kann das Resultat von fetchall zu Speicherproblemen führen. 
Wir können die Datensätze des Resultats auch mit einer for-Schleife über den cursor erhalten.

In [30]:
connection = sq.connect("lagerverwaltung.db")  
cursor = connection.cursor()

sql = "SELECT * FROM kunden"

cursor.execute(sql)
for zeile in cursor:
    print(zeile)
connection.close()


(12, 'Heinz Elhurg', 'Turnhallenstr. 1, 3763 Sporthausen')
(57, 'Markus Altbert', 'Kämperweg 24, 2463 Duisschloss')
(64, 'Steve Apple', 'Podmacstr 2, 7467 Iwarhausen')
